In [ ]:
import os 
os.environ["HF_HOME"] = "/your/path/hf_cache"
from datasets import load_dataset 

    

In [ ]:

v7w_data = load_dataset('json', 
                         data_files="/your/path/visual7w/dataset.json", 
                         split="train")

In [ ]:
from PIL import Image
idx = 0
# print(v7w_data[idx].keys()) 
# # dict_keys(['images', 'version', 'dataset'])
# print(v7w_data[idx]["images"].keys()) 
# dict_keys(['filename', 'image_id', 'qa_pairs', 'split'])
for qa_pair in v7w_data[idx]["images"]['qa_pairs']:
    print(qa_pair['question'], qa_pair['answer'])

In [ ]:
import json
with open("/your/path/visdial/dataset.json", "r") as f:
    visdial_data = json.load(f)

# Find the proper questions
Qs = []
for i, ques in enumerate(visdial_data["data"]["questions"]):
    if "weather" in ques:
        Qs.append(i)

In [ ]:
dial_data = []
for dialog in visdial_data["data"]["dialogs"]:
    for qa in dialog["dialog"]:
        if qa["question"] in Qs:
            QA = {}
            QA["question"] = visdial_data["data"]["questions"][qa["question"]]
            QA["answer"] = visdial_data["data"]["answers"][qa["answer"]]
            QA["image_id"] = dialog["image_id"]
            dial_data.append(QA)
            
bad_data = []
with open("/your/path/visdial/bad_data.txt", "r") as f:
    for line in f.readlines():
        bad_data.append(line.strip())

In [ ]:
bad_data_coco = []
with open("/your/path/visdial/bad_data_coco.txt", "r") as f:
    for line in f.readlines():
        bad_data_coco.append(line.strip())
with open("/your/path/COCO-2017/annotations/captions_val2017.json", "r") as f:
    coco_data = json.load(f)


In [ ]:
count = 0
sunnycount = 0
rainycount = 0
snowycount = 0
cloudycount = 0
duskcount = 0
stormycount = 0
coldcount = 0
warmcount = 0
all_other_answers = []

# There are instances with multiple weather queries
# First for Visual7W
for item in v7w_data:
    if item["images"]["split"] != 'val': continue
    for qa_pair in item["images"]["qa_pairs"]:
        if "weather" in qa_pair["question"]:
            if "sunny" in qa_pair["answer"].lower() or "clear" in qa_pair["answer"].lower() or "sunshine" in qa_pair["answer"].lower(): 
                sunnycount += 1
            elif "cloudy" in qa_pair["answer"].lower() or "overcast" in qa_pair["answer"].lower() or "misty" in qa_pair["answer"].lower(): 
                cloudycount += 1
            elif "rainy" in qa_pair["answer"].lower() or "drizzly" in qa_pair["answer"].lower() or "rain" in qa_pair["answer"].lower():
                rainycount += 1
            elif "snow" in qa_pair["answer"].lower(): snowycount += 1
            elif "stormy" in qa_pair["answer"].lower(): stormycount += 1
            elif "warm" in qa_pair["answer"].lower(): 
                print(qa_pair['question'], qa_pair['answer'])
                warmcount += 1
            elif "cold" in qa_pair["answer"].lower() or "chilly" in qa_pair["answer"].lower(): coldcount += 1
            else:
                # print(qa_pair['question'], qa_pair['answer'])
                all_other_answers.append(qa_pair['answer'])
                continue
            count += 1
        elif "cloudy" in qa_pair["answer"].lower():
            cloudycount += 1
            count += 1
        
# Then for Visual Dialog
for item in reversed(dial_data):
    if item["image_id"] in bad_data: dial_data.remove(item)
    if "sunny" in item["answer"].lower() or "clear" in item["answer"].lower() or "sunshine" in item["answer"].lower(): 
        sunnycount += 1
    elif "cloudy" in item["answer"].lower() or "overcast" in item["answer"].lower() or "misty" in item["answer"].lower(): 
        cloudycount += 1
    elif "rainy" in item["answer"].lower() or "drizzly" in item["answer"].lower() or "rain" in item["answer"].lower():
        rainycount += 1
    elif "snow" in item["answer"].lower(): snowycount += 1
    elif "stormy" in item["answer"].lower(): stormycount += 1
    elif "warm" in item["answer"].lower(): warmcount += 1
    elif "cold" in item["answer"].lower() or "chilly" in item["answer"].lower(): coldcount += 1
    else:
        all_other_answers.append(item['answer'])
        continue
    count += 1
    
# Finally for val2017
for anno in reversed(coco_data["annotations"]):
    if anno["image_id"] in bad_data_coco:
        coco_data["annotations"].remove(anno)
    if "sunny" in anno["caption"].lower() or "clear" in anno["caption"].lower() or "sunshine" in anno["caption"].lower():
        sunnycount += 1
    elif "cloudy" in anno["caption"].lower() or "overcast" in anno["caption"].lower() or "misty" in anno["caption"].lower():
        cloudycount += 1
    elif "rainy" in anno["caption"].lower() or "drizzly" in anno["caption"].lower() or "rain" in anno["caption"].lower():
        rainycount += 1
    elif "snow" in anno["caption"].lower(): snowycount += 1
    elif "stormy" in anno["caption"].lower(): stormycount += 1
    elif "warm" in anno["caption"].lower(): warmcount += 1
    elif "cold" in anno["caption"].lower() or "chilly" in anno["caption"].lower(): coldcount += 1
    else:
        all_other_answers.append(anno['caption'])
        continue
    count += 1
    

print(count, sunnycount, rainycount, snowycount, cloudycount, stormycount, warmcount, coldcount)


In [ ]:
from wordcloud import WordCloud

# Word cloud
import matplotlib.pyplot as plt

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(all_other_answers))

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Weather categories
# Create a dictionary to hold the image IDs for each weather category
categories = {"sunny":[], "cloudy":[], "rainy":[], "snowy":[], "stormy": [], "warm": [], "cold": [],
              "clear":[], "misty": [], "overcast":[], "drizzly": [], "chilly": []}
images_1, images_2, images_3 = [], [], []
for item in v7w_data:
    if item["images"]["split"] != 'val': continue
    gt = None
    for qa_pair in item["images"]["qa_pairs"]:
        if "weather" in qa_pair["question"]:
            if "sunny" in qa_pair["answer"].lower() or "sunshine" in qa_pair["answer"].lower(): 
                gt = "sunny"
            elif "clear" in qa_pair["answer"].lower():
                gt = "clear"
            elif "cloudy" in qa_pair["answer"].lower(): 
                gt = "cloudy"
            elif "overcast" in qa_pair["answer"].lower():
                gt = "overcast" 
            elif "misty" in qa_pair["answer"].lower(): 
                gt = "misty"
            elif "rainy" in qa_pair["answer"].lower() or "rain" in qa_pair["answer"].lower():
                gt = "rainy"
            elif "drizzly" in qa_pair["answer"].lower():
                gt = "drizzly"
            elif "snow" in qa_pair["answer"].lower(): gt = "snowy"
            elif "stormy" in qa_pair["answer"].lower(): gt = "stormy"
            elif "warm" in qa_pair["answer"].lower(): gt = "warm"
            elif "cold" in qa_pair["answer"].lower(): gt = "cold"
            elif "chilly" in qa_pair["answer"].lower(): gt = "chilly"
        elif "cloudy" in qa_pair["answer"].lower():
            gt = "cloudy"

    if gt != None: 
        categories[gt].append(item['images']['filename'])
        images_1.append(item['images']['filename'])
        
for item in dial_data:
    if "sunny" in item["answer"].lower() or "sunshine" in item["answer"].lower(): 
        categories["sunny"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    elif "clear" in item["answer"].lower():
        categories["clear"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    elif "cloudy" in item["answer"].lower(): 
        categories["cloudy"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    elif "overcast" in item["answer"].lower():
        categories["overcast"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    elif "misty" in item["answer"].lower(): 
        categories["misty"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    elif "rainy" in item["answer"].lower() or "rain" in item["answer"].lower():
        categories["rainy"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    elif "drizzly" in item["answer"].lower():
        categories["drizzly"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    elif "snow" in item["answer"].lower(): 
        categories["snowy"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    elif "stormy" in item["answer"].lower(): 
        categories["stormy"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    elif "warm" in item["answer"].lower(): 
        categories["warm"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    elif "cold" in item["answer"].lower(): 
        categories["cold"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    elif "chilly" in item["answer"].lower(): 
        categories["chilly"].append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
        images_2.append("VisualDialog_val2018_"+(item['image_id']).zfill(12)+".jpg")
    
for item in coco_data["annotations"]:
    if "sunny" in item["caption"].lower() or "sunshine" in item["caption"].lower(): 
        categories["sunny"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")
    elif "clear" in item["caption"].lower():
        categories["clear"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")
    elif "cloudy" in item["caption"].lower():
        categories["cloudy"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")
    elif "overcast" in item["caption"].lower():
        categories["overcast"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")
    elif "misty" in item["caption"].lower():
        categories["misty"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")
    elif "rainy" in item["caption"].lower() or "rain" in item["caption"].lower():
        categories["rainy"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")
    elif "drizzly" in item["caption"].lower():
        categories["drizzly"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")
    elif "snow" in item["caption"].lower(): 
        categories["snowy"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")
    elif "stormy" in item["caption"].lower(): 
        categories["stormy"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")
    elif "warm" in item["caption"].lower(): 
        categories["warm"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")
    elif "cold" in item["caption"].lower(): 
        categories["cold"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")
    elif "chilly" in item["caption"].lower(): 
        categories["chilly"].append((item['image_id']).zfill(12)+".jpg")
        images_3.append((item['image_id']).zfill(12)+".jpg")

In [ ]:
# Check the repetition between sets
import imagehash

def are_similar_hash(image_path1, image_path2, hash_size=8, similarity_threshold=10):
    """
    Use hash to show similarity

    Args:
        image_path1 (str)
        image_path2 (str)
        hash_size (int)
        similarity_threshold (int)

    Returns:
        bool
    """
    try:
        img1 = Image.open(image_path1).resize((hash_size, hash_size), Image.Resampling.LANCZOS).convert('L')
        img2 = Image.open(image_path2).resize((hash_size, hash_size), Image.Resampling.LANCZOS).convert('L')

        hash1 = imagehash.phash(img1, hash_size=hash_size)
        hash2 = imagehash.phash(img2, hash_size=hash_size)

        if (hash1 - hash2) < similarity_threshold:
            return True
        else:
            return False
    except FileNotFoundError:
        print("Error: One or both image files not found.")
        print(f"File not found: {image_path1} or {image_path2}")
        return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

def are_similar_ahash(image_path1, image_path2, hash_size=8, similarity_threshold=10):
    """
    Use the average hash to decide the similarity
    """
    try:
        img1 = Image.open(image_path1).resize((hash_size, hash_size), Image.Resampling.LANCZOS).convert('L')
        img2 = Image.open(image_path2).resize((hash_size, hash_size), Image.Resampling.LANCZOS).convert('L')

        hash1 = imagehash.average_hash(img1, hash_size=hash_size)
        hash2 = imagehash.average_hash(img2, hash_size=hash_size)

        if (hash1 - hash2) < similarity_threshold:
            return True
        else:
            return False
    except FileNotFoundError:
        print("Error: One or both image files not found.")
        return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

In [ ]:
for file in images_2:
    for another in images_1:
        hash = are_similar_ahash("you/path/images"+file, "you/path/images"+another)
        ahash = are_similar_ahash("you/path/images"+file, "you/path/images"+another)
        if hash or ahash:
            for key in categories:
                if file in categories[key]:
                    categories[key].remove(file)
                    break
        
for file in images_3:
    for another in images_1:
        hash = are_similar_ahash("you/path/images"+file, "you/path/images"+another)
        ahash = are_similar_ahash("you/path/images"+file, "you/path/images"+another)
        if hash or ahash:
            for key in categories:
                if file in categories[key]:
                    categories[key].remove(file)
                    break
    for another in images_2:
        hash = are_similar_ahash("you/path/images"+file, "you/path/images"+another)
        ahash = are_similar_ahash("you/path/images"+file, "you/path/images"+another)
        if hash or ahash:
            for key in categories:
                if file in categories[key]:
                    categories[key].remove(file)
                    break

In [ ]:
import random

new_data = []
exclusive = {"sunny": ["cloudy", "misty", "overcast", "rainy", "drizzly", "snowy", "stormy", "cold", "chilly"], 
             "clear":  ["cloudy", "misty", "overcast", "rainy", "drizzly", "snowy", "stormy", "cold", "chilly"], 
             "misty":  ["sunny", "clear", "snowy"], 
             "overcast": ["sunny", "clear", "snowy"],
             "cloudy": ["sunny", "clear", "snowy"],
             "rainy": ["sunny", "clear", "warm", "snowy"], 
             "drizzly": ["sunny", "clear", "warm", "snowy"], 
             "stormy": ["sunny", "clear", "warm", "snowy"],
             "snowy": ["sunny", "clear", "rainy", "drizzly", "stormy", "warm"],
             "warm": ["rainy", "stormy", "snowy", "cold", "chilly", "overcast", "misty"],
             "cold": ["sunny", "clear", "warm"],
             "chilly": ["sunny", "clear", "warm"]}
for label in categories:
    for data in categories[label]:
        new_item = {}
        if label == "warm":
            continue # We exclude warm here
        if label == "overcast":
            new_item['qry_text'] = f"Find me an everyday image that is taken in an {label} day.\n"
        else:
            new_item['qry_text'] = f"Find me an everyday image that is taken in a {label} day.\n" 
        new_item['qry_img_path'] = ''
        new_item['tgt_text'] = "<|image_1|> Represent the given image."
        new_item['tgt_img_path'] = [data]
        all_paths = []
        for place in categories:
            if place in exclusive[label]:
                all_paths.extend(categories[place])
        new_item['tgt_img_path'].extend(random.sample(all_paths, 99))
        new_data.append(new_item)


In [ ]:
import json

with open('mix_weather_retrieval.json', 'w') as f:
    json.dump(new_data, f, indent=4)

In [ ]:
# Test whether we can load it using load_dataset
new_v7w_data = load_dataset('json', 
                      data_files='mix_weather_retrieval.json',
                      split="train")

In [ ]:
idx = 1
print(new_v7w_data[idx].keys())
print(new_v7w_data[idx]['qry_text'])
img = Image.open("/your/path/visual7w/images/"+new_v7w_data[idx]['tgt_img_path'][60]).convert("RGB")
img